In [5]:
import cv2
import toml
import sys
import os

sys.path.append(os.path.abspath("data_management"))
sys.path.append(os.path.abspath("onnx_models"))

import data_preprocessing as dprep
import data_postprocessing as dpostp
import onnx_inference as onnx_inf

%matplotlib inline

In [6]:
yolo_cfg = toml.load('yolov5n.toml')
imgsz = yolo_cfg['input_data']['imgsz']
conf_thres = yolo_cfg['predictor']['conf_thres']
iou_thres = yolo_cfg['predictor']['iou_thres']
classes = yolo_cfg['predictor']['classes']
if not classes:
    classes = None
onnx_model_path = yolo_cfg['onnx_model_path']
visualize = yolo_cfg['visualize']
out_img_path = yolo_cfg['out_img_path']
input_source = yolo_cfg['source']

In [7]:
stride, names, session, output_names = onnx_inf.load_model(onnx_model_path)

In [8]:
try:
    cap.release()
    cap.destroyAllWindows()
except Exception as e:
    print(e)
    pass

if input_source == 'webcam':
    cap = cv2.VideoCapture(0) 
else:
    cap = cv2.VideoCapture(input_source)

frame_count = 0
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check that frame is not empty
    if ret:
        orig_img = frame.copy()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        img = dprep.img_preprocess(img, imgsz, stride)

        predictions = onnx_inf.model_inf(img, session, output_names)

        predictions = dpostp.yolo_nms(predictions, conf_thres, iou_thres, classes)

        for pred in predictions:

            pred[:, :4] = dpostp.scale_boxes((imgsz, imgsz), pred[:, :4], orig_img.shape).round()

            for p in pred:
                print({"object":names[int(p[5])], "x":int(p[0]), "y":int(p[1]), "width":int(p[2])-int(p[0]), "height":int(p[3])-int(p[1])})

                if visualize:
                    cv2.rectangle(orig_img, (int(p[0]), int(p[1])), (int(p[2]), int(p[3])), (0, 255, 0), 2)
                    cv2.putText(orig_img, names[int(p[5])], (int(p[0]), int(p[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if visualize:
            cv2.imwrite(os.path.join(out_img_path, f'{frame_count}.png'), orig_img)
            frame_count += 1

    else:
        break
        
try:
    cap.release()
    cap.destroyAllWindows()
except Exception as e:
    print(e)
    pass

'cv2.VideoCapture' object has no attribute 'destroyAllWindows'
{'object': 'person', 'x': 458, 'y': 5, 'width': 182, 'height': 429}
{'object': 'person', 'x': 306, 'y': 17, 'width': 332, 'height': 455}
{'object': 'person', 'x': 278, 'y': 26, 'width': 360, 'height': 446}
{'object': 'person', 'x': 225, 'y': 13, 'width': 408, 'height': 463}
{'object': 'person', 'x': 229, 'y': 20, 'width': 407, 'height': 453}
{'object': 'person', 'x': 225, 'y': 33, 'width': 414, 'height': 436}
{'object': 'person', 'x': 234, 'y': 33, 'width': 404, 'height': 441}
{'object': 'person', 'x': 244, 'y': 25, 'width': 393, 'height': 452}
{'object': 'person', 'x': 246, 'y': 40, 'width': 391, 'height': 438}
{'object': 'person', 'x': 251, 'y': 55, 'width': 383, 'height': 425}
{'object': 'person', 'x': 258, 'y': 38, 'width': 381, 'height': 442}
{'object': 'person', 'x': 262, 'y': 44, 'width': 378, 'height': 431}
{'object': 'person', 'x': 265, 'y': 62, 'width': 369, 'height': 413}
{'object': 'person', 'x': 265, 'y': 63, '

KeyboardInterrupt: 